In [16]:
import math

# Define the stat sheet for offense
class OffensiveStats(object):
    def __init__(self, name):
        self.name = name
        
    def set_weapon_skill(self, ws):
        self.ws = ws
        
    def set_strength(self, strength):
        self.strength = strength
        
    def set_balistic_skill(self, bs):
        self.bs = bs
        
    def set_ap(self, ap):
        self.ap = ap
        
    def set_wounds(self, w):
        self.w = w
        
    def mean_attacks_to_kill(self, defensive_stats, is_ranged = True):
        attack = self.ws
        if is_ranged:
            attack = self.bs
            
        # probability of hitting
        prob_hit = (6.0 - attack) / 6.0
        
        # probability of wounding
        # str >= 2* tough => 2+
        # str > tough => 3+
        # str = tough => 4+
        # str < tough => 5+
        # 2*str <= tough => 6+
        t = defensive_stats.toughness
        s = self.strength
        
        wound_on = 5.0
        if s >= 2*t:
            wound_on = 2.0
        elif s > t:
            wound_on = 3.0
        elif s == t:
            wound_on = 4.0
        elif 2*s < t:
            wound_on = 6.0
        
        prob_wound = (7.0 - wound_on) / 6.0
        
        # probability of saving against the wound
        # determine save
        save = defensive_stats.save - self.ap
        if defensive_stats.has_invul:
            if defensive_stats.invul < save:
                save = defensive_stats.invul
        prob_save = (7.0 - save) / 6.
        
        attacks_to_wound = int(1.0 / (prob_hit * prob_wound * (1.0 - prob_save)))
        wound_to_kill = int(math.ceil(defensive_stats.wounds / self.w))
        attacks_to_kill = attacks_to_wound * wound_to_kill
        print "Probability of " + self.name + " hitting " + defensive_stats.name + ": " + str(prob_hit) + "\nProbability of wounding: " + str(prob_wound) + "\nProbability of saving: " + str(prob_save) + "\nTotal attacks to kill: " + str(attacks_to_kill)
        return attacks_to_kill
        
class DefensiveStats(object):
    def __init__(self, name):
        self.name = name
        self.has_invul = False
        
    def set_save(self, save):
        self.save = save
    
    def set_invul(self, invul):
        self.invul = invul
        self.has_invul = True
        
    def set_toughness(self, toughness):
        self.toughness = toughness
        
    def set_wounds(self, wounds):
        self.wounds = wounds
        
    

In [17]:
wych = OffensiveStats("wych")
wych.set_weapon_skill(3.0)
wych.set_balistic_skill(3.0)
wych.set_strength(3.0)
wych.set_ap(0.0)
wych.set_wounds(1.0)

necron = DefensiveStats("necron")
necron.set_toughness(4.0)
necron.set_save(4.0)
necron.set_wounds(1.0)

In [18]:
wych.mean_attacks_to_kill(necron)

Probability of wych hitting necron: 0.5
Probability of wounding: 0.333333333333
Probability of saving: 0.5
Total attacks to kill: 12


12